# Download YOLOv3 and SimplePose model from MXNET model zoo
- Choose Data Science kernel and ml.t3.medium instance
- Run following code to have models download
- The downloaded models are saved in ~/.mxnet/models folder. You should be able to see them from the Image Terminal
- Go into the ~/.mxnet/models folder and use following command to create a tar ball file including these two models  
`
cd ~/.mxnet/models
tar -cvzf model.tar.gz simple_pose_resnet18_v1b-f63d42ac.params yolo3_mobilenet1.0_coco-66dbbae6.params
`
- Upload the model.tar.gz file into an S3 bucket with following command. Modify S3_BUCKET_NAME and MODEL_FILE_KEY_PREFIX to your own choice.  
`aws s3 cp model.tar.gz s3://{S3_BUCKET_NAME}/{MODEL_FILE_KEY_PREFIX}/model.tar.gz`


In [ ]:
from gluoncv import model_zoo, data, utils
detector = model_zoo.get_model('yolo3_mobilenet1.0_coco', pretrained=True)
pose_net = model_zoo.get_model('simple_pose_resnet18_v1b', pretrained=True)

# Build with original MXNet Docker Image
- Run following code to create a SageMaker model with the model file
- A SageMaker endpoint will be created based on the MXNet model file

In [ ]:
import numpy as np
import json
import boto3
from time import gmtime, strftime
client = boto3.client('sagemaker-runtime')
import sagemaker
from sagemaker.mxnet import MXNetModel
bucket_name = 'S3_BUCKET_NAME'
s3_prefix = 'MODEL_FILE_KEY_PREFIX'
model = MXNetModel(
    entry_point='entrypoint.py',
    model_data='s3://{}/{}/model.tar.gz'.format(bucket_name, s3_prefix),
    framework_version='1.6.0',
    py_version='py3',
    source_dir='inference_src',
    role=sagemaker.get_execution_role(),
    name='Extract-pose-keypoints-yolo3-simplepose-'+ strftime('%Y-%m-%d-%H-%M-%S', gmtime())
)

In [ ]:
ep_name = 'Extract-pose-keypoints-yolo3-simplepose-'+ strftime('%Y-%m-%d-%H-%M-%S', gmtime())
predictor = model.deploy(initial_instance_count=1, 
                        instance_type='ml.m5.4xlarge',
                        endpoint_name=ep_name)

# Inference sample code

In [ ]:
# Test with SageMaker runtime client
test_image = 'images/soccer.png'
img = open(test_image, 'rb').read()
response = client.invoke_endpoint(
    EndpointName=ep_name,
    Body=img,
    ContentType='application/x-image')

result = response["Body"].read().decode("utf-8")
detections = json.loads(result)
print (detections)

In [ ]:
# Test with SageMaker predictor
from sagemaker.predictor import Predictor

object_detector = Predictor(endpoint_name=ep_name)
results = object_detector.predict(img,initial_args={"ContentType": "image/jpeg"})
detections = json.loads(results)
detections

# Show keypoints

In [ ]:
from matplotlib import pyplot as plt
img = plt.imread(test_image)
print(img.shape)
fig,ax = plt.subplots(1, dpi=120)
ax.imshow(img)
for dots in detections['1']['qcresult']:
    for idx, dot in enumerate(dots):
        if idx==0: continue
        x,y,xb,yb = dot
        x = x*img.shape[1]
        y = y*img.shape[0]
           
        rect = plt.plot(x,y,'b.')
ax.axis('off')
plt.show()

# Delete the endpoint after use

In [ ]:
boto3.client('sagemaker').delete_endpoint(EndpointName=ep_name)